In [59]:
import recordlinkage as rl
import pandas as pd
import os
import random as ran
from sklearn.model_selection import GridSearchCV

In [5]:
schema_path = "../schema_matching/csv/schema_final.csv"

In [6]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True) # Per evitare che gli indici siano letti come decimanli (i.e. 1.0 invece di 1)

# Preprocessamento

In [7]:
from recordlinkage.preprocessing import *

## Cleaning dei dati ...

In [8]:
schema_cleaned = schema # creo una copia per non modificare lo schema iniziale

In [9]:
# Cleaning dei dati iterando per riga
for i in range(len(schema.index)):
    row = schema.iloc[i]
    row_series = row.squeeze() # BOO
    nome = pd.Series(row["name"])
    try:
        nome_cleaned = clean(nome, lowercase=True, replace_by_none='', replace_by_whitespace='', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
    except:
        print(nome)
    # Si evita il cleaning di caratteri relative alle valute presenti nei datasets
    row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$€£¥₩₽₹฿₪¤¢₺₱]+', replace_by_whitespace=r'[\\-\\_]', strip_accents=None, remove_brackets=False, encoding='utf-8', decode_error='strict')
    schema_cleaned.iloc[i] = row_series_cleaned
    schema_cleaned.iloc[i]["name"] = nome_cleaned[0]
print(schema_cleaned)

                                            name        country market cap  \
0                             berkshire hathaway  united states        NaN   
1                                           icbc          china        NaN   
2       saudi arabian oil company (saudi aramco)   saudi arabia        NaN   
3                                 jpmorgan chase  united states        NaN   
4                        china construction bank          china        NaN   
...                                          ...            ...        ...   
188552                                   gazprom         russia        NaN   
188553                                   rosseti         russia        NaN   
188554                                 nornickel         russia        NaN   
188555                                 severstal         russia        NaN   
188556                                      ozon            usa        NaN   

       founded year employees                        industry s

# Occorrenze token per riga

In [10]:
# value_occurence(schema)

# Blocking

In [11]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned)
print(candidate_links)

MultiIndex([(  2966,      0),
            ( 12966,      0),
            ( 12966,   2966),
            ( 30538,      0),
            ( 30538,   2966),
            ( 30538,  12966),
            ( 34398,      0),
            ( 34398,   2966),
            ( 34398,  12966),
            ( 34398,  30538),
            ...
            (176795, 176576),
            (176851, 176580),
            (176729, 176591),
            (176697, 176602),
            (176751, 176606),
            (177007, 176616),
            (176863, 176642),
            (176934, 176671),
            (176984, 176796),
            (177004, 176877)],
           length=549188)


# Labeling

In [12]:
keys = ['name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'stock', 'share price', 'city', 'address', 'website']

clear = lambda: os.system('clear')

In [13]:
n_sampled_matches = 450
choices = ran.choices(candidate_links, k=n_sampled_matches)

lKeys = {key : f"l_{key}" for key in keys}
rKeys = {key : f"r_{key}" for key in keys}

idxKeys = ["id_1", "id_2"]

choices_column = [*idxKeys, *lKeys.values(), *rKeys.values()] # id_1, id_2, l_attr, r_attr
choices_df = pd.DataFrame(columns=choices_column) # 
for choice in choices:
    lRow = schema_cleaned.iloc[[choice[0]]]
    rRow = schema_cleaned.iloc[[choice[1]]]

    lRow = lRow.rename(columns=lKeys).reset_index(drop=True)
    rRow = rRow.rename(columns=rKeys).reset_index(drop=True)

    idxs = pd.DataFrame({idxKeys[0]: [choice[0]], idxKeys[1]: [choice[1]]})
    row = pd.concat([idxs, lRow, rRow], axis=1)
    choices_df = pd.concat([choices_df, row], axis=0)

# choices_df.reset_index(inplace=True)
choices_df.set_index(idxKeys)

,,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,l_revenue,l_stock,...,r_employees,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website
id_1,id_2,,,,,,,,,,,,,,,,,,,,,
105142,94476,great wall enterprise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpswww.dachan.com
88269,46095,tenaris,NaN,NaN,NaN,19399,NaN,NaN,paolo rocca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$37.56,NaN,NaN,NaN
164790,35342,alinma bank,s. arabia,$17.07 b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$8.90,NaN,NaN,NaN
83347,39792,mesoblast,australia,$0.47 b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$3.28,NaN,NaN,NaN
53428,37911,casio,NaN,NaN,april 1946 76 years ago 194604 as kashio seisa...,NaN,electronics,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$10.10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88581,35830,universal health services,NaN,NaN,NaN,89400,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$147.88,NaN,NaN,NaN
116210,53834,stellantis,netherlands,$48.84 b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,automotive production systems,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177651,133225,eversource energy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,$9.86 b,NaN,NaN,NaN,NaN,NaN


In [14]:
choices_path = "./csv/choices.csv"
choices_df.to_csv(choices_path, index=None)

In [15]:
choices_path_gallo = "./csv/gallo.csv"
choices_path_gatto = "./csv/gatto.csv"
choices_path_moli = "./csv/moli.csv"

sampled_matches_per_person = n_sampled_matches//3

choices_df_gallo = choices_df.iloc[:sampled_matches_per_person]
print(len(choices_df_gallo))

choices_df_gatto = choices_df.iloc[sampled_matches_per_person:2*sampled_matches_per_person]
print(len(choices_df_gatto))

choices_df_moli = choices_df.iloc[2*sampled_matches_per_person:]
print(len(choices_df_moli))

choices_df_gallo.to_csv(choices_path_gallo, index=None)
choices_df_gatto.to_csv(choices_path_gatto, index=None)
choices_df_moli.to_csv(choices_path_moli, index=None)

150
150
150


# Comparing

In [16]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [17]:
features

name   country  founded year  industry  sector  address  city  \
2966   0        1.0  0.230769      0.750000  0.225000     0.0      0.0   0.0   
12966  0        1.0  0.000000      0.250000  0.000000     0.0      0.0   0.0   
       2966     1.0  0.000000      0.333333  0.000000     0.0      0.0   0.0   
30538  0        1.0  1.000000      0.750000  0.173913     0.0      0.0   0.0   
       2966     1.0  0.230769      1.000000  0.400000     0.0      0.0   0.0   
...             ...       ...           ...       ...     ...      ...   ...   
177007 176616   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   
176863 176642   1.0  1.000000      0.500000  1.000000     0.0      0.0   0.0   
176934 176671   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   
176984 176796   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   
177004 176877   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   

                    ceo  
2966   0       0.666667  
12966  0       0.480000  
       2966    0.560000  
30538  0       0.571429  
       2966    0.666667  
...                 ...  
177007 176616  0.000000  
176863 176642  0.000000  
176934 176671  0.000000  
176984 176796  0.000000  
177004 176877  0.000000  

[549188 rows x 8 columns]

In [18]:
features.describe()

,name,country,founded year,industry,sector,address,city,ceo
count,549188.0,549188.000000,549188.000000,549188.000000,549188.000000,549188.000000,549188.000000,549188.00000
mean,1.0,0.428896,0.042812,0.035206,0.042351,0.011631,0.004850,0.04938
std,0.0,0.487790,0.193214,0.175677,0.200951,0.088958,0.069441,0.20354
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [19]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.000000       314
5.972222         1
5.968750         1
5.966102         1
5.961538         1
             ...  
1.017857         1
1.016667         1
1.016393         1
1.011820         1
1.000000    245256
Length: 5788, dtype: int64

In [20]:
record_linked = features[features.sum(axis=1) > 1]

In [21]:
print(record_linked)

               name   country  founded year  industry  sector  address  city  \
2966   0        1.0  0.230769      0.750000  0.225000     0.0      0.0   0.0   
12966  0        1.0  0.000000      0.250000  0.000000     0.0      0.0   0.0   
       2966     1.0  0.000000      0.333333  0.000000     0.0      0.0   0.0   
30538  0        1.0  1.000000      0.750000  0.173913     0.0      0.0   0.0   
       2966     1.0  0.230769      1.000000  0.400000     0.0      0.0   0.0   
...             ...       ...           ...       ...     ...      ...   ...   
177007 176616   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   
176863 176642   1.0  1.000000      0.500000  1.000000     0.0      0.0   0.0   
176934 176671   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   
176984 176796   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   
177004 176877   1.0  1.000000      1.000000  1.000000     0.0      0.0   0.0   

                    ceo  
2966   0     

In [30]:
golden_path_gallo = "./csv/golden_links_gallo.csv"
golden_path_gatto = "./csv/golden_links_gatto.csv"
golden_path_moli = "./csv/golden_links_moli.csv"

golden_gallo = pd.read_csv(golden_path_gallo, index_col=None)
golden_gatto = pd.read_csv(golden_path_gatto, index_col=None)
golden_moli = pd.read_csv(golden_path_moli, index_col=None)

golden = pd.concat([golden_gallo, golden_gatto, golden_moli], axis=0)

golden_path = "./csv/golden_links.csv"

golden = golden.reset_index(drop=True)
golden.to_csv(golden_path, index=None)

,id_1,id_2,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,...,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website,label
0,105142,94476,great wall enterprise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpswww.dachan.com,1
1,88269,46095,tenaris,NaN,NaN,NaN,19399.0,NaN,NaN,paolo rocca,...,NaN,NaN,NaN,NaN,NaN,$37.56,NaN,NaN,NaN,0
2,34579,15142,target,usa,NaN,NaN,NaN,NaN,NaN,NaN,...,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,129177,53681,microsoft,NaN,NaN,1975.0,NaN,it internet software services,NaN,NaN,...,information technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,116955,1058,jde peet's,netherlands,$15.21 b,NaN,NaN,NaN,NaN,NaN,...,food drink,NaN,fabien jacques simon,$8.3b,NaN,NaN,NaN,NaN,NaN,1
5,80390,294,bank of ningbo,china,$29.59 b,NaN,NaN,NaN,NaN,NaN,...,banking,NaN,meng bo luo,$13.7b,NaN,NaN,NaN,NaN,NaN,0
6,129972,97911,apollo global management,NaN,NaN,1990.0,NaN,diversified financials,NaN,NaN,...,NaN,NaN,marc rowan,NaN,NaN,NaN,NaN,NaN,httpswww.apollo.com,1


In [57]:
golden = pd.read_csv(golden_path, index_col=[0,1])
golden_links = golden[golden["label"]==1].index

5

# Classification (Unsupervised)

In [22]:
ECM = rl.ECMClassifier(binarize=0.6)
result = ECM.fit_predict(record_linked)
print("Possible record linked size:", len(record_linked))
print("Record linked size:", len(result))
print(f"Removed {len(record_linked) - len(result)} possible links")

Possible record linked size: 303932
Record linked size: 27940
Removed 275992 possible links


# Evaluation (Unsupervised)

### Confusion matrix

In [ ]:
rl.confusion_matrix(golden_links, result)

### Precision

In [ ]:
rl.precision(golden_links, result)

### Recall

In [ ]:
rl.recall(golden_links, result)

### Accuracy

In [ ]:
rl.accuracy(golden_links, result)

### F-score

In [ ]:
rl.fscore(golden_links, result)

### Specificity

In [ ]:
rl.specificity(golden_links, result)

# Classification (Supervised)

In [62]:
rlSvm = rl.SVMClassifier()
params = {}
gridSearch = GridSearchCV(estimator=rlSvm, param_grid=params)

# Evaluation (Supervised)

### Confusion matrix

In [ ]:
rl.confusion_matrix(golden_links, )

### Precision

In [ ]:
rl.precision(golden_links,)

### Recall

In [ ]:
rl.recall(golden_links,)

### Accuracy

In [ ]:
rl.accuracy(golden_links,)

### F-score

In [ ]:
rl.fscore(golden_links,)

### Specificity

In [ ]:
rl.specificity(golden_links,)